In [ ]:
%%capture

from IPython.core.display import HTML
with open('style.css') as file:
    css = file.read()
HTML(css)

%run s04_engine_setup.ipynb

# AI Engine

## Add first evaluation function

The next engine 
is the first one
to evaluate chess positions 
to find better moves. 
Instead of creating the `AbsoluteEvaluationEngine` as one block of code,
a slightly different approach will be taken here: First all class attributes and methods will be shown and then the class is assembled.
To avoid confusions and global namespace pollution all methods are prefixed with the class name. 
In the class later they won't have the prefix and therefore are in explanations as well as in other methods referred to without their prefix.


A simple way of evaluating a move 
is based on absolute values for each piece.
The dictionary `PIECE_VALUES` associates a chess piece type with a value as commonly defined in {cite}`Capablanca2006` and other books.

In [ ]:
import chess

AbsoluteEvaluationEngine_PIECE_VALUES = {
    chess.PAWN: 1,
    chess.KNIGHT: 3,
    chess.BISHOP: 3,
    chess.ROOK: 5,
    chess.QUEEN: 9,
    chess.KING: 0
}

The evaluation
of the game 
is determined by summing up the piece values of all the white pieces
and subtracting the piece values of all the black pieces. 
A positive score therefore is good for White whereas a negative score is good for Black. 
This is implemented in the method `_absolute_heuristic`,
which takes a board as input 
and returns an integer representing this evaluation. 

In [ ]:
def AbsoluteEvaluationEngine__absolute_heuristic(
    self, board: chess.Board
) -> int:
    """Calculate the value of all pieces on the board"""

    whitePositions = chess.SquareSet(board.occupied_co[chess.WHITE])
    whiteScore = sum(
        self.PIECE_VALUES[board.piece_type_at(square)]
        for square in whitePositions
    )

    blackPositions = chess.SquareSet(board.occupied_co[chess.BLACK])
    blackScore = sum(
        self.PIECE_VALUES[board.piece_type_at(square)]
        for square in blackPositions
    )

    return whiteScore - blackScore

To have an incentive to win,
the value of a winning board 
is valued with 100.
As a draw has no advantage for either side
it is valued with 0.
For better readability 
these values are also stored in constants.

In [ ]:
AbsoluteEvaluationEngine_VALUE_CHECKMATE = 100
AbsoluteEvaluationEngine_VALUE_DRAW = 0

There exist multiple draw conditions, 
so for ease of use, 
they are combined into one function, 
which is added to the `chess.Board` class afterwards.
This form of dynamic class modification is called monkey patching in the python community 
and is often used as a technique to patch third party code {cite}`plone:glossary`.

In [ ]:
def is_draw(self) -> bool:
    """Function to check for all automatically enforced draw conditions"""
    return self.is_stalemate() or self.is_insufficient_material(
    ) or self.is_fivefold_repetition() or self.is_seventyfive_moves()


chess.Board.is_draw = is_draw

Having declared this helper function and the values for end conditions, the next function `_evaluate_move` calculates the score of a single move. 
The given move - or rather the resulting next board - is evaluated by first checking
for checkmate or draw, or otherwise by using the previously implemented `_absolute_heuristic` method.

In [ ]:
def AbsoluteEvaluationEngine__evaluate_move(
    self, board: chess.Board, move: chess.Move
) -> ScoredMove:
    """Evaluate a single move using _absolute_heuristic"""
    whitesTurn = board.turn is chess.WHITE

    board.push(move)
    if board.is_checkmate():
        score = self.VALUE_CHECKMATE if whitesTurn else -self.VALUE_CHECKMATE
    elif board.is_draw():
        score = self.VALUE_DRAW
    else:
        score = self._absolute_heuristic(board)
    board.pop()

    return ScoredMove(score, move)

Next, the `analyse` method can be defined to score all legal moves using the `_evaluate_move` method.

The scored moves are then shuffled and, afterwards, sorted by their score, 
creating a different order of moves having the same score, 
depending on the state of the RNG.

By default, the python `sort` method sorts from lowest to highest. 
Therefore, the first move is the best for Black, unless the sorting order is reversed.

In [ ]:
import random


def AbsoluteEvaluationEngine_analyse(self,
                                     board: chess.Board) -> list[ScoredMove]:
    """Analyse method using _absolute_heuristic"""
    nextMoves = [self._evaluate_move(board, move) for move in board.legal_moves]

    random.shuffle(nextMoves)

    whitesTurn = board.turn is chess.WHITE
    nextMoves.sort(reverse=whitesTurn)

    return nextMoves

The `play` method will simply return the best move from `analyse` as an `chess.engine.PlayResult`. 
In case of having multiple best values, the taken move is random as the `analyse` shuffles the list.

In [ ]:
def AbsoluteEvaluationEngine_play(
    self, board: chess.Board
) -> chess.engine.PlayResult:
    bestScoredMove = self.analyse(board)[0]
    return chess.engine.PlayResult(move=bestScoredMove.move, ponder=None)

Having all functions and attributes defined, the `AbsoluteEvaluationEngine` class can be assembled.

In [ ]:
class AbsoluteEvaluationEngine(Engine):
    """Chess engine using the absolute value of the chessboard to evalute moves"""
    # Class attributes
    VALUE_CHECKMATE = AbsoluteEvaluationEngine_VALUE_CHECKMATE
    VALUE_DRAW = AbsoluteEvaluationEngine_VALUE_DRAW
    PIECE_VALUES = AbsoluteEvaluationEngine_PIECE_VALUES

    # Helper methods
    _absolute_heuristic = AbsoluteEvaluationEngine__absolute_heuristic
    _evaluate_move = AbsoluteEvaluationEngine__evaluate_move

    # Public methods
    play = AbsoluteEvaluationEngine_play
    analyse = AbsoluteEvaluationEngine_analyse

The `AbsoluteEnhancedRandomEngine` strategy is to capture everything the engine can. 
Additionally, it is also able to find mate in one. 
Therefore, it has very good chances to win against the `RandomEngine`.

In [ ]:
random.seed(42)
board_absolute = chess.Board()
board_absolute = playGame(
    board_absolute,
    AbsoluteEvaluationEngine(),
    RandomEngine(),
    displayBoard=False
)
IPython.display.display(board_absolute)
print(board_absolute.outcome())

To verfify the correctness the `_absolute_heuristic` its output is checked against some famous chess games.

In [ ]:
random.seed(42)
engine = AbsoluteEvaluationEngine()

# Topalov, Veselin (2740) vs. Shirov, Alexei (2710)
SHIROV_SACRIFICE = "8/8/4kpp1/3p1b2/p6P/2B5/6P1/6K1 b - - 0 47"
board = chess.Board(SHIROV_SACRIFICE)
# IPython.display.display(board)
assert engine._absolute_heuristic(board) == -2, "Evaluation of position is not -2"

# Evgeny Yuryevich Vladimirov vs. Vladimir Viktorovich Epishin
VLADIMIROV_THUNDERBOLT = "r4k1r/1b2bPR1/p4n2/3p4/4P2P/1q2B2B/PpP5/1K4R1 w - - 0 26"
board = chess.Board(VLADIMIROV_THUNDERBOLT)
# IPython.display.display(board)
assert engine._absolute_heuristic(board) == -10, "Evaluation of position is not -10"

Additionally, a special chess position was constructed to test the `analyse` method. In this position, different moves such as capturings and promotions are possible.

In [ ]:
random.seed(42)
PROMOTION_POSITION = "Kn2rn1k/1p2P3/8/8/8/8/8/8 w - - 0 1"
board = chess.Board(PROMOTION_POSITION)
# IPython.display.display(board)
assert engine._absolute_heuristic(board) == -11, "Evaluation of position is not -11"
result = engine.analyse(board)
assert result == [ScoredMove(0, chess.Move.from_uci("e7f8q")),
                  ScoredMove(-4, chess.Move.from_uci("e7f8r")),
                  ScoredMove(-6, chess.Move.from_uci("e7f8b")),
                  ScoredMove(-6, chess.Move.from_uci("e7f8n")),
                  ScoredMove(-10, chess.Move.from_uci("a8b7")),
                  ScoredMove(-11, chess.Move.from_uci("a8b7"))], f'{result} is not correct'